<a href="https://colab.research.google.com/github/andreaaraldo/machine-learning-for-networks/blob/master/08.predictive-maintenance/Predictive-maintenance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We first need to revert to an older version of scikit-learn, for a compatibility issue with `scikit-survival`, the library we are going to use for our Survival Analysis

In [1]:
!pip uninstall scikit-learn -y

Uninstalling scikit-learn-0.22:
  Successfully uninstalled scikit-learn-0.22


In [2]:
pip install scikit-learn==0.22

  Using cached https://files.pythonhosted.org/packages/2e/d0/860c4f6a7027e00acff373d9f5327f4ae3ed5872234b3cbdd7bcb52e5eff/scikit_learn-0.22-cp36-cp36m-manylinux1_x86_64.whl


In [3]:
!pip install scikit-survival



In [0]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored

In [30]:
! wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/08.predictive-maintenance/dataset/transformed/test_set.csv
! wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/08.predictive-maintenance/dataset/transformed/training_set.csv

--2020-05-19 23:05:46--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/08.predictive-maintenance/dataset/transformed/test_set.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4397297 (4.2M) [text/plain]
Saving to: ‘test_set.csv’

test_set.csv        100%[===================>]   4.19M  22.1MB/s    in 0.2s    

2020-05-19 23:05:47 (22.1 MB/s) - ‘test_set.csv’ saved [4397297/4397297]

--2020-05-19 23:05:50--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/08.predictive-maintenance/dataset/transformed/training_set.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubuserc

In [35]:
! head training_set.csv

setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,remaining_duration,failure_observed
0.45977011494252873,0.16666666666666663,0.0,0.0,0.18373493975903443,0.4068018312622641,0.30975692099932317,0.0,1.0,0.72624798711756,0.24242424242402194,0.10975500314098952,0.0,0.3690476190476204,0.6332622601279212,0.20588235294098922,0.1996078026628112,0.36398614851866284,0.0,0.3333333333333357,0.0,0.0,0.7131782945736447,0.7246616956641816,191,1
0.6091954022988506,0.25,0.0,0.0,0.2831325301204686,0.45301940265969165,0.3526333558406485,0.0,1.0,0.628019323671495,0.2121212121210192,0.10024230458584071,0.0,0.38095238095238315,0.7654584221748308,0.2794117647058556,0.16281349984518556,0.41131204309350267,0.0,0.3333333333333357,0.0,0.0,0.6666666666666643,0.731013532173435,190,0
0.2528735632183908,0.75,0.0,0.0,0.34337349397591765,0.36952256376716974,0.370526671168129,0.0,1.0,0.7101449275362341,0.2727272727270247,0.1400430763708158,0.0,0.25000000000000355,0.79

In [0]:
df_trn = pd.read_csv("training_set.csv")
df_test = pd.read_csv("training_set.csv")

In [32]:
df_trn.head()

,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,remaining_duration,failure_observed
0,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,1.0,0.726248,0.242424,0.109755,0.0,0.369048,0.633262,0.205882,0.199608,0.363986,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,1
1,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,1.0,0.628019,0.212121,0.100242,0.0,0.380952,0.765458,0.279412,0.162813,0.411312,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,0
2,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,1.0,0.710145,0.272727,0.140043,0.0,0.250000,0.795309,0.220588,0.171793,0.357445,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,0
3,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,1.0,0.740741,0.318182,0.124518,0.0,0.166667,0.889126,0.294118,0.174889,0.166603,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,1
4,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,1.0,0.668277,0.242424,0.149960,0.0,0.255952,0.746269,0.235294,0.174734,0.402078,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,1


In [33]:
X_trn = df_trn.drop(columns=["remaining_duration", "failure_observed"])
y_trn = df_trn[["failure_observed", "remaining_duration"]]

y_trn

,failure_observed,remaining_duration
0,1,191
1,0,190
2,0,189
3,1,188
4,1,187
...,...,...
20626,1,4
20627,1,3
20628,1,2
20629,1,1


In [23]:
print(f'Number of samples: {len(y_trn)}')
print(f'Number of right censored samples: {len(y_trn.query("failure_observed == False"))}')
print(f'Percentage of right censored samples: {100*len(y_trn.query("failure_observed == False"))/len(y_trn):.1f}%')

Number of samples: 20631
Number of right censored samples: 2026
Percentage of right censored samples: 9.8%


The dataset is already scaled and all the columns are numerical. We do not need to do pre-processing for that.

In [25]:
model = CoxPHSurvivalAnalysis()
model.fit(X_trn, y_trn.values)

ValueError: ignored